In [1]:
import pandas as pd
import plotly.express as px

list_of_files = ['jan',
                'feb',
                'mar',
                'apr',
                'apr23_to_may05',
                'may07_to_may12',
                'may14_to_may19'
                ]

headings = {
    'median_income_jan': 'jan',
    'median_income_feb': 'feb',
    'median_income_mar': 'mar',
    'median_income_apr': 'apr',
    'median_income_apr23_to_may05': 'apr23_to_may05',
    'median_income_may07_to_may12': 'may07_to_may12',
    'median_income_may14_to_may19': 'may14_to_may19'
    }

In [2]:
# get data from file
def plot_month(month):
    month_data = pd.read_csv('data_processing_income\clean data\\' + month + '.csv')
    
    # define specific value for median so that colours can be plotted. Also, correct for the different ranges in files. Should automate this!!

# '''
    medians_range_to_value = {
        'Less Than $5,000': 2500,
        '5,000 To 7,499': 6500,
        '7,500 To 9,999': 8750,
        '10,000 To 12,499': 11500,
        '12,500 To 14,999': 13750,
        '15,000 To 19,999': 17500,
        '20,000 To 24,999': 22500,
        '25,000 To 29,999': 27500,
        '30,000 To 34,999': 32500,
        '35,000 To 39,999': 37500,
        '40,000 To 49,999': 45000,
        '50,000 To 59,999': 55000,
        '60,000 To 74,999': 67500,
        '75,000 To 99,999': 87500,
        '100,000 To 149,999': 125000,
        '150,000 or More': 150000,
        '    Less than $25,000': 12500,
        '    $25,000 - $34,999': 30000,
        '    $35,000 - $49,999': 42500,
        '    $50,000 - $74,999': 62500,
        '    $75,000 - $99,999': 87500,
        '    $100,000 - $149,999': 125000,
        '    $150,000 - $199,999': 175000,
        '    $200,000 and above': 200000
    }
# '''

    # replace range with specific value
    month_data['median_income_' + month] = month_data['median_income_' + month].replace(medians_range_to_value).astype(float)
    
    return month_data


In [ ]:
# plot_month('apr')
plot_month('may07_to_may12')

In [ ]:
# reference https://wellsr.com/python/creating-python-choropleth-maps-with-plotly/
# plots each file separately

def plot_map(month):
    
    month_data = plot_month(month)

    fig = px.choropleth(month_data,
                        locations = 'abbreviations',
                        color = 'median_income_' + month,
                        color_continuous_scale = 'algae',
                        hover_name = 'Selected Geographies',
                        locationmode = 'USA-states'
    )

    fig.update_layout(
        title_text = 'Median Incomes for ' + month,
        geo_scope = 'usa',
    )

    fig.show()
    return

In [ ]:
plot_map('jan')

In [ ]:
# load pre-prepped template
# rename columns into months, then append columns into a new dataframe

def combine_data():

    template = pd.read_csv('data_processing_income\clean data\\template.csv')
    # print(template)
    
    combined = template

    for i in range(len(list_of_files)):
        # combined[list_of_files[i]] = plot_month(list_of_files[i]).columns[3]
        combined[list_of_files[i]] = plot_month(list_of_files[i]).iloc[:, 3]
    
    # print(combined)
    
    return combined

In [ ]:
combine_data().to_csv(f'data_processing_income\clean data\combined.csv', index = False)

In [3]:
# append list of values in 1 income column
# create new column with months

def append_data():
    
    appended = pd.DataFrame()

    for i in range(len(list_of_files)):
        # get column names
        old_column_name = plot_month(list_of_files[i]).columns[3]
        
        # rename column into general income
        renamed_plot = plot_month(list_of_files[i]).rename(columns = {old_column_name: 'median_income'})

        # add a column with month information
        renamed_plot['month'] = old_column_name

        # replace old columns with just the months
        renamed_plot = renamed_plot.replace({'month':headings})

        if i == 0:
            appended = renamed_plot

        else:
            appended = appended.append(renamed_plot, ignore_index = True)

        # appended['month'] = plot_month(list_of_files[i]).columns[3]

        # appended['median_income'] = plot_month(list_of_files[i]).columns[3]
    
    return appended

In [ ]:
# save a copy of the dataframe to see it
append_data().to_csv('test.csv')

In [4]:
# plot files together

def plot_time(data):

    fig = px.choropleth(data,
                        locations = 'abbreviations',
                        color = 'median_income',
                        animation_frame = 'month',
                        animation_group = 'FIPS_code',
                        color_continuous_scale = 'algae',
                        hover_name = 'Selected Geographies',
                        locationmode = 'USA-states',
                        scope = 'usa',
                        title = 'Median incomes for 2020'
    )

    fig.show()
    return

In [5]:
appended = append_data()
plot_time(appended)